In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 8.0)
import sys
import timeit
from pynx.ptycho.bragg import *
from pynx.processing_unit import default_processing_unit
from pynx.wavefront import Wavefront, ImshowRGBA, PropagateNearField
from pynx.utils.math import smaller_primes
from silx.io.specfile import SpecFile
import fabio
from scipy.ndimage.measurements import center_of_mass
default_processing_unit.use_gpu()

/users/zatterin/Envs/py3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Computing FFT speed for available CUDA GPU[ranking by fft, fft_shape=(16, 400, 400)]:
                                        GeForce GTX 1060 6GB: 6072Mb , 184.40 Gflop/s
                                                Quadro M2000: 4037Mb ,  69.81 Gflop/s
Searching available OpenCL GPU[ranking by fft, fft_shape=(16, 400, 400)]:
                          GeForce GTX 1060 6GB [NVIDIA CUDA]: 6072Mb [max alloc.: 1518Mb], 208.36 Gflop/s
                                  Quadro M2000 [NVIDIA CUDA]: 4037Mb [max alloc.: 1009Mb],  59.42 Gflop/s
Using OpenCL GPU: GeForce GTX 1060 6GB


True

In [11]:
# Bragg ptycho parameters
specfile= "/data/id01/inhouse/tao/IH-HC-3372/spec/PTO8STO3_pty.spec"
scan0, scan1 = 1, 2  # scan range
image_prefix= "/data/id01/inhouse/tao/IH-HC-3372/mpx/PTO8STO3_pty/data_mpx4_%05d.edf.gz"

In [21]:
# Read spec scans
s = SpecFile(specfile)['{0}.1'.format(scan0)]
h = s.scan_header_dict
vs = {}

# read scan energy
nrj = float(h['UMONO'].split('mononrj=')[1].split('ke')[0])
wavelength = 12.384 / nrj * 1e-10
print("Reading nrj from spec data: nrj=%6.3fkeV, wavelength=%6.3fA" % (nrj, wavelength*1e10))

# read det distance
detector_distance = float(h['UDETCALIB'].split('stance_CC=')[1].split(',')[0])
print("Detector distance: %6.3fm"% detector_distance)

# read other 
vs={'eta':np.deg2rad(s.motor_position_by_name('eta')),
       'phi':np.deg2rad(s.motor_position_by_name('phi')), 
       'del':np.deg2rad(s.motor_position_by_name('del')),
       'del':np.deg2rad(s.motor_position_by_name('del')),
       'nu':np.deg2rad(s.motor_position_by_name('nu')),
       'mpx4inr':s.data_column_by_name('mpx4inr').astype(np.int),
       'pix': s.data_column_by_name('pix') ,'piy': s.data_column_by_name('piy'),
       'mpx4int': s.data_column_by_name('mpx4int'),
       'roi2': s.data_column_by_name('roi2'),
       'roi4': s.data_column_by_name('roi4'),
       'filter': float(h['UFILT1'])
      }

Reading nrj from spec data: nrj= 7.685keV, wavelength= 1.611A
Detector distance:  0.506m


In [23]:
vs['eta']

0.6768385792036459

In [24]:
# Angular scan parameters
filter_f = 3.9
eta = vs['eta']
# veta = np.array([ vs[i]['eta'] for i in range(scan0,scan1)])
# iobssum = np.array([ vs[i]['mpx4int'].sum() * filter_f**vs[i]['filter'] for i in range(scan0,scan1)])
# roi4sum = np.array([ vs[i]['roi4'].sum() * filter_f**vs[i]['filter'] for i in range(scan0,scan1)])
# roi2sum = np.array([ vs[i]['roi2'].sum() * filter_f**vs[i]['filter'] for i in range(scan0,scan1)])
# etastep = (veta[-1] - veta[0])/(len(veta)-1)
# print("Angular range= %5.3f°, step=%6.4f°, %d spirals"%(np.rad2deg(veta[-1] - veta[0]), np.rad2deg(etastep), npsi))
# # Intensity vs angle
# %matplotlib inline
# plt.semilogy(np.rad2deg(veta), iobssum)
# #ipsi0 = int(round(center_of_mass(np.array(roi2sum))[0]))
# ipsi0 = len(iobssum)//2
# print(ipsi0)

In [ ]:
# Load all images for the central spiralscan
iscan0 = scan0 + ipsi0
print('iscan0=',iscan0)
iobsc = None
ii = 0
t0= timeit.default_timer()
for i in vs[iscan0]['mpx4inr']:
    if (i - vs[iscan0]['mpx4inr'][0]) % 20 == 0:
        sys.stdout.write('%d ' % (i - vs[iscan0]['mpx4inr'][0]))
        sys.stdout.flush()
    frame = np.fliplr(fabio.open(image_prefix % i).data)
    if iobsc is None:
        iobsc = np.empty((len(vs[iscan0]['mpx4inr']), frame.shape[0], frame.shape[1]), dtype=frame.dtype)
    iobsc[ii] = frame
    ii += 1
print("\n")
dt = timeit.default_timer() - t0
print('Time to read all frames: %4.1fs [%5.2f Mpixel/s]' % (dt, iobsc.size / 1e6 / dt))

In [ ]:
# Find center of diffraction
plt.imshow(np.log10(iobsc.sum(axis=0)))
y0, x0 = center_of_mass(iobsc.sum(axis=0))
print("Center of mass: y0=%5.1f, x0=%5.1f"%(y0,x0))
ny0, nx0 = iobsc.shape[-2:]
ix0 = int(round(x0))
iy0 = int(round(y0))

if True:
    # Manually correct position
    #ix0 -= 35
    iy0 = 100
plt.scatter([ix0],[iy0])  # To check visually

In [ ]:
# Size of crop array
nx = 2 * min(ix0, nx0 - ix0)
ny = 2 * min(iy0, ny0 - iy0)

# Not too large
if nx > 300:
    nx = 300
if ny > 300:
    ny = 300

npsi0 = scan1-scan0  # +1 ?
npsi = 2 * min(ipsi0, npsi0 - ipsi0)
# crop to max fft size
npsi, ny, nx = smaller_primes((npsi, ny, nx), maxprime=default_processing_unit.max_prime_fft, required_dividers=(2,))
veta = veta[:npsi]

In [ ]:
# Select area - too many frames !
#plt.plot(vs[ic]['pix'],iobsc.sum(axis=(1,2)))
v = iobsc.sum(axis=(1,2))
xp = vs[iscan0]['pix']
yp = vs[iscan0]['piy']
zp = 0 * xp
%matplotlib inline
ax=plt.scatter(xp,yp,40,v/v.max(), vmin=0,vmax=1, cmap=plt.cm.Greys, edgecolors='face')
plt.gca().set_aspect(1)
plt.xlabel('X (µm)')
plt.ylabel('Y (µm)')
print(zp.shape)

In [ ]:
vpos = np.arange(len(xp))  #np.nonzero((xp>94.5)*(xp<95.3)*(yp>111)*(yp<111.7))[0]
npos = len(vpos)
print("Number of scan positions selected: %d"%(npos))

In [ ]:
# Now load all the data
iobs = np.empty((npos,npsi,ny,nx),dtype=np.float32)
ii = 0
t0= timeit.default_timer()
print("%d frames to read:"%(npos*npsi))
for j in range(len(vpos)):
    ipos = vpos[j]
    for ipsi in range(npsi):
        i = vs[iscan0-npsi//2+ipsi]['mpx4inr'][ipos]
        corr = filter_f ** vs[iscan0-npsi//2+ipsi]['filter']
        if ii % 20 == 0:
            sys.stdout.write('%d ' % (ii))
            sys.stdout.flush()
        frame = np.fliplr(fabio.open(image_prefix % i).data) * corr
        iobs[j, ipsi] = frame[iy0-ny//2:iy0+ny//2,ix0-nx//2:ix0+nx//2]
        ii += 1
print("\n")
dt = timeit.default_timer() - t0
print('Time to read all frames: %4.1fs [%5.2f Mpixel/s]' % (dt, iobs.size / 1e6 / dt))

In [ ]:
%matplotlib inline
plt.imshow(np.log10(iobs[6,:].sum(axis=(0))))
#plt.imshow(np.log10(iobs.sum(axis=(0,-2))))
#plt.imshow(np.log10(iobs[15,16]))

In [ ]:
# Experiment parameters
pixel_size_detector = 55e-6
psi_step = etastep
print("Cropped angular range: %6.3f, step=%6.4f"%(np.rad2deg(psi_step)*npsi, np.rad2deg(psi_step)))

In [ ]:
## Ptycho data

# Use only one stack - if memory allows it !
default_processing_unit.cl_stack_size = npos

# detector parameters
nu = vs[scan0]['nu']
delta = vs[scan0]['del']
eta0 = veta.mean()
detector = {'geometry': 'psic', 'delta': delta, 'nu': nu, 'pixel_size': pixel_size_detector,
            'distance': detector_distance, 'rotation_axis': 'eta', 'rotation_step': psi_step}
zs, ys, xs = zp[vpos] * 1e-6, yp[vpos] * 1e-6, xp[vpos] * 1e-6

ys *= np.sin(veta.mean())  # (we don't care about movement along z)

print('del= %5.2f nu =%5.2f eta0=%5.2f eta_step=%6.4f'%(np.rad2deg(detector['delta']),np.rad2deg(detector['nu']), np.rad2deg(eta0),np.rad2deg(detector['rotation_step'])))
# Create empty data
data = BraggPtychoData(iobs=iobs, positions=(zs, -ys, -xs), mask=None, wavelength=wavelength, detector=detector)

In [ ]:
# Import existing probe from 2D ptycho - only the first mode will be used
d = np.load("/data/id01/inhouse/edo/paul_ptycho/ResultsScan0031/latest.npz") # Siemens star
#d = np.load("ResultsScan0096/latest.npz")  # use the bragg ptycho scan
dpr = np.fft.fftshift(d['probe'],axes=(-2,-1))  # just flip the x,y axes - not the modes
#
pr = Wavefront(d=dpr, z=0, pixel_size=d['pixelsize'], wavelength=wavelength)
%matplotlib inline
pr = ImshowRGBA() * PropagateNearField(0e-6) * pr

In [ ]:
# Create main Bragg Ptycho object
p = BraggPtycho(probe=pr, data=data, support=None)
pxyz = p.voxel_size_object()
print("Object voxel size: %6.2fnm x %6.2fnm x %6.2fnm" % (pxyz[0] * 1e9, pxyz[1] * 1e9, pxyz[2] * 1e9))

In [ ]:
%matplotlib inline
# Create starting object
x, y, z = p.get_xyz(domain='object', rotation=('x', eta0))
obj0 = (abs(y) < 62e-9)
#obj0 = obj0 * np.random.uniform(0,1,obj0.shape)
p.set_obj(obj0)

p = ShowObj() * p

# Set a support larger than the object
sup = (abs(y) < 100e-9)
p.set_support(sup)


In [ ]:
%matplotlib notebook
p = DM(calc_llk=10, show_obj_probe=20) ** 200 * p

In [ ]:
p = AP(calc_llk=10, show_obj_probe=10) ** 40 * p
p = ML(calc_llk=10, show_obj_probe=10) ** 40 * p

In [ ]:
p=FreePU()*p
#p=ShowObj()*AP()**40*p

In [ ]:
print("cos(<phi>=%6.2f)=%6.4f"%(np.rad2deg(vphi.mean()), np.cos(vphi.mean())))

In [ ]:
np.savez_compressed("tmp.npz",obj0=obj0.astype(np.int8),obj=p.get_obj())

In [ ]:
np.savez_compressed("iobs16.npz",iobs=iobs[16])